In [1]:
import mujoco
import numpy as np
import matplotlib.pyplot as plt
import mediapipe as mp
import cv2
import sys
sys.path.append("../utils")
from mujoco_parser import MuJoCoParserClass
from util import rpy2r
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("cv2:[%s]"%(cv2.__version__))
print ("mediapipe:[%s]"%(mp.__version__))

xml_path = '../asset/object/floor_sky.xml'
env = MuJoCoParserClass(name='SMPLH-Rig',rel_xml_path=xml_path,VERBOSE=False)

env.init_viewer(viewer_title='Joint Pos',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=180,distance=1.0,elevation=-21,lookat=[0.02,-0.03,0.8])
env.reset()

hand_joint_idx_pairs = [[0,1],[1,2],[2,3],[3,4],
                        [0,5],[5,6],[6,7],[7,8],
                        [5,9],[9,10],[10,11],[11,12],
                        [9,13],[13,14],[14,15],[15,16],
                        [0,17],[13,17],[17,18],[18,19],[19,20]]

cv2:[4.9.0]
mediapipe:[0.10.9]


In [2]:
valid_cams = []
for i in range(8):
    cap = cv2.VideoCapture(i)
    if cap is None or not cap.isOpened():
        print('Warning: unable to open video source: ', i)
    else:
        valid_cams.append(i)

print(valid_cams)

[0]


OpenCV: out device of bound (0-0): 1
OpenCV: camera failed to properly initialize!
OpenCV: out device of bound (0-0): 2
OpenCV: camera failed to properly initialize!
OpenCV: out device of bound (0-0): 3
OpenCV: camera failed to properly initialize!
OpenCV: out device of bound (0-0): 4
OpenCV: camera failed to properly initialize!
OpenCV: out device of bound (0-0): 5
OpenCV: camera failed to properly initialize!
OpenCV: out device of bound (0-0): 6
OpenCV: camera failed to properly initialize!
OpenCV: out device of bound (0-0): 7
OpenCV: camera failed to properly initialize!


In [3]:
cap = cv2.VideoCapture(0)

# Initialize MediaPipe hand model
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode        = False,
    max_num_hands            = 2,
    model_complexity         = 1,
    min_detection_confidence = 0.25,
    min_tracking_confidence  = 0.25,
)
mp_drawing = mp.solutions.drawing_utils

while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        break

    img = cv2.flip(img, 1)
    img.flags.writeable = False
    res = hands.process(img)

    img_with_res = img.copy()
    if res.multi_hand_landmarks:
        for hand_landmarks in res.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                img_with_res,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
            )

    n_landmark = 21 # fixed
    if res.multi_hand_world_landmarks: # if hands are detected
        n_detected_hand = len(res.multi_hand_world_landmarks) # number of detected hand(s)
        joint_xyzs_list = []
        for h_idx in range(n_detected_hand):
            joint_xyzs = np.zeros((n_landmark,3)) # [21 x 3]
            for l_idx in range(n_landmark): # for each landmark
                joint_xyzs[l_idx,0] = res.multi_hand_world_landmarks[h_idx].landmark[l_idx].x
                joint_xyzs[l_idx,1] = res.multi_hand_world_landmarks[h_idx].landmark[l_idx].y
                joint_xyzs[l_idx,2] = res.multi_hand_world_landmarks[h_idx].landmark[l_idx].z
            joint_xyzs_list.append(joint_xyzs)
    
    else: # if no hands are detected
        n_detected_hand = 0
        joint_xyzs_list = []

    # cv2.imshow('original_video', img)
    cv2.imshow('hands_3d', img_with_res)
    
    # Hand rotation offset
    R = rpy2r(np.radians([0,0,-90]))@rpy2r(np.radians([0,90,0]))
    # Plot hand joints
    colors = [(1,0,0,1),(0,0,1,1),(0,0,0,0)]
    xyz_offsets = [np.array([0,-0.25,1.0]),np.array([0,0.25,1.0]),np.array([0,0,0])]
    for h_idx in range(n_detected_hand): # for each hand
        joint_xyzs = joint_xyzs_list[h_idx]@R # ndarray [21 x 3]
        n_joint = 21
        for j_idx in range(n_joint):
            joint_xyz = joint_xyzs[j_idx,:]+xyz_offsets[h_idx] # [3]
            rgba = colors[h_idx]
            env.plot_sphere(p=joint_xyz,r=0.005,rgba=rgba)

        for hand_joint_idx_pair in hand_joint_idx_pairs:
            idx_fr,idx_to = hand_joint_idx_pair[0],hand_joint_idx_pair[1]
            env.plot_cylinder_fr2to(
                p_fr = joint_xyzs[idx_fr,:]+xyz_offsets[h_idx],
                p_to = joint_xyzs[idx_to,:]+xyz_offsets[h_idx],
                r    = 0.003,
                rgba = [0.7,0.7,0.7,1],
            )

    env.plot_T(p=np.zeros(3),R=np.eye(3,3),PLOT_AXIS=True,axis_len=0.1,axis_width=0.005)

    env.render()

    key = cv2.waitKey(1)
    if key==27:
        break

cap.release()
env.close_viewer()

I0000 00:00:1711017722.632069       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
